<a href="https://colab.research.google.com/github/psriraj17/MachineTranslation/blob/main/MT_anythinggoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [160]:
import collections
from tqdm import tqdm
import string
import helper
import numpy as np
#import project_tests as tests
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from nltk.translate.bleu_score import corpus_bleu , sentence_bleu
import re

In [161]:
def load_data(file):
    all_data = []
    descript = 'Reading ' + file

    f = open(file, 'r', encoding='utf-8')
    full_text = f.read()

    cur_sent = []

    for line in tqdm(full_text.split('\n'), desc=descript):
        if line == '<s>':
            cur_sent = ''
            continue
        if line in '()':
            continue
        if line == '</s>':
            all_data.append(cur_sent)
            continue
        else:
            cur_sent += line.lower()
            cur_sent += ' '

    return all_data


#source_data = helper.load_data('/content/drive/MyDrive/Colab Notebooks/train-05/train-source.txt')
source_sentences = load_data("/content/drive/MyDrive/Colab Notebooks/train-05/train-source.txt")
target_sentences = load_data("/content/drive/MyDrive/Colab Notebooks/train-05/train-target.txt")
#taregt_data = helper.load_data('/content/drive/MyDrive/Colab Notebooks/train-05/train-target.txt')


print('\nDataset Loaded',source_sentences[0:10])

Reading /content/drive/MyDrive/Colab Notebooks/train-05/train-source.txt: 100%|██████████| 925535/925535 [00:00<00:00, 2142340.84it/s]
Reading /content/drive/MyDrive/Colab Notebooks/train-05/train-target.txt: 100%|██████████| 910805/910805 [00:00<00:00, 2144501.23it/s]


Dataset Loaded ['cinnte go leór , thiocfadh dóbhtha bás a fhagháil ar imeall an phuill udaí . ', 'bhí sé follasach go rabh an poll sin ag foscladh ar an fhairrge ar dhóigh éigin , ná líonadh agus thráigheadh an t-uisce ann . ', "d'fhéadfadh siad bás ' fhagháil ar a bhruach agus na cuirp imtheacht ar an lán mhara amach fríd an phluais . ", 'thiocfadh dóbhtha fosta lámh a chur ina mbás féin , a ghabháil de léim isteach sa pholl ghalach a bhí i n-iarthar an dara taibhlidh - poll mar bhéadh coire de uisce ghalach ann . ', "' na dhiaidh sin , bhí rud éigin do-chreidte agus leamh in gach teóir de'n bheirt . ", 'nuair a scríobh siad an litir sin bhí siad araon i sláinte mhaith agus tréan bidh aca . ', 'bhí fhios againn i gceart nach rabh díothbhail ar bith ortha , na diomaoite de chuid bonnóg chathail tháinig muid ar an áit taiscthe a bhí aca - scealpach bheag eachar dhá charraic - agus arán agus spólaí de chaoir-fheóil shaillte ann . ', "b'fhuras a fheiceáil cá dtáinig an chaoir-fheóil . ",

In [162]:
source_words_counter = collections.Counter([word for sentence in source_sentences for word in sentence.split()])
target_words_counter = collections.Counter([word for sentence in target_sentences for word in sentence.split()])

print('{} Source words.'.format(len([word for sentence in source_sentences for word in sentence])))
print('{} unique Source words.'.format(len(source_words_counter)))
print('10 Most common words in the Source dataset:')
print('"' + '" "'.join(list(zip(*source_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Target words.'.format(len([word for sentence in target_sentences for word in sentence])))
print('{} unique Target words.'.format(len(target_words_counter)))
print('10 Most common words in the Target dataset:')                
print('"' + '" "'.join(list(zip(*target_words_counter.most_common(10)))[0]) + '"')

4127703 Source words.
31035 unique Source words.
10 Most common words in the Source dataset:
"." "a" "an" "," "agus" "ar" """ "bhí" "na" "sé"

3939531 Target words.
25735 unique Target words.
10 Most common words in the Target dataset:
"." "a" "an" "," "agus" "ar" "bhí" """ "ag" "sé"


In [163]:
def tokenize(x):
    
    # TODO: Implement
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

In [164]:
def pad(x, length=None):
    
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

In [165]:
def preprocess(x, y):
    
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_source_sentences, preproc_target_sentences, source_tokenizer, target_tokenizer =\
    preprocess(source_sentences, target_sentences)
    
max_source_sequence_length = preproc_source_sentences.shape[1]
max_target_sequence_length = preproc_target_sentences.shape[1]
source_vocab_size = len(source_tokenizer.word_index)
target_vocab_size = len(target_tokenizer.word_index)

print('Data Preprocessed')
print("Max source sentence length:", max_source_sequence_length)
print("Max target sentence length:", max_target_sequence_length)
print("source vocabulary size:", source_vocab_size)
print("target vocabulary size:", target_vocab_size)


Data Preprocessed
Max source sentence length: 215
Max target sentence length: 213
source vocabulary size: 27496
target vocabulary size: 24967


In [166]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [167]:
def model_final(input_shape, output_sequence_length, source_vocab_size, target_vocab_size):
  

    model = keras.models.Sequential()
    model.add(Embedding(input_dim=source_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(target_vocab_size,activation='softmax')))
    learning_rate = 0.001
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

print('Final Model Loaded')

Final Model Loaded


In [168]:
def final_predictions(x, y, x_tk, y_tk):
  
    
    tmp_X = preproc_source_sentences
    model = model_final(tmp_X.shape,
                        preproc_target_sentences.shape[1],
                        len(source_tokenizer.word_index)+1,
                        len(target_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_target_sentences, batch_size = 100, epochs = 10, validation_split = 0.2)
 
    return model

In [ ]:
model = final_predictions(preproc_source_sentences, preproc_target_sentences, source_tokenizer, target_tokenizer)

Epoch 1/10
362/362 [==============================] - 396s 1s/step - loss: 0.8297 - accuracy: 0.9205 - val_loss: 0.4951 - val_accuracy: 0.9321
Epoch 2/10
 98/362 [=======>......................] - ETA: 4:18 - loss: 0.5441 - accuracy: 0.9251

In [ ]:
def simple_fixes(s):
  s = s.replace('sg','sc')
  s = s.replace('sd','st')
  s = s.replace('sb','sp')
  s = s.replace('eó','eo')
  s = s.replace('thch','ch')
  s = re.sub(r'll([rst])', r'l\1', s)
  return s

In [ ]:
def anything_goes_translation(sentence , test_source_tokenizer, test_target_tokenizer):
  #sentence = sentence.translate(str.maketrans('', '', string.punctuation))
  #print("\n",sentence)
  #sentence = [test_source_tokenizer.word_index[word] for word in sentence.split()]
  sentence = pad_sequences([sentence], maxlen=preproc_source_sentences.shape[-1], padding='post')
  predictions = model.predict(sentence, len(sentence))
  
  y_id_to_word = {value: key for key, value in test_target_tokenizer.word_index.items()}
  y_id_to_word[0] = ''
  
  source_sent = (' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
  #print("source sentences",source_sent)
  return [simple_fixes(s) for s in source_sent] 

In [ ]:
def evaluate():
  testsource = load_data("/content/drive/MyDrive/Colab Notebooks/test-05/test-source.txt")
  testtarget = load_data('/content/drive/MyDrive/Colab Notebooks/test-05/test-target.txt')

  test_source_sentences , test_source_tokenizer = tokenize(testsource)
  test_target_sentences , test_target_tokenizer = tokenize(testtarget)

  anything_goes_hypotheses = []
  for sentence in test_source_sentences:
    anything_goes_hypotheses.append(anything_goes_translation(sentence,test_source_tokenizer, test_target_tokenizer))
  print(anything_goes_hypotheses)
  return (corpus_bleu(testtarget,anything_goes_hypotheses))

In [ ]:
evaluate()